In [1]:
import numpy as np
import pandas as pd
import csv
from time import time

# plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from datetime import datetime
import matplotlib.pyplot as plt
import category_encoders as ce

In [2]:
train = pd.read_csv('train.csv')

In [3]:
#Capitales de los Estados de México
capitales=['Aguascalientes','Mexicali','La Paz','Campeche','Tuxtla Gutiérrez','Chihuahua','Saltillo','Colima','Durango','Guanajuato','Chilpancingo de los Bravo','Pachuca','Guadalajara','Toluca','Morelia','Cuernavaca','Tepic','Monterrey','Oaxaca de Juárez','Puebla','Querétaro','Chetumal','San Luis Potosí','Culiacán','Hermosillo','Villahermosa','Victoria','Tlaxcala','Xalapa','Mérida','Zacatecas','Cuauhtémoc']

In [4]:
#Función que establece si una ciudad se encuentra en la capital del estado
def EstaEnCapital(Ciudad):
    for capital in capitales:
        if(Ciudad==capital):
            return 1
    return 0

In [5]:
train['Esta_Capital']=train['ciudad'].apply(EstaEnCapital)

In [6]:
train['Esta_condominio']=train.titulo.str.contains('condominio')
train['Esta_loma']=train.titulo.str.contains('loma|lomas')
train['Esta_bosque']=train.titulo.str.contains('bosque')
train['Esta_country']=train.titulo.str.contains('country')
train['Esta_lago']=train.titulo.str.contains('lago')
train['Esta_residencial']=train.titulo.str.contains('residencial')
train['Esta_penthouse']=train.titulo.str.contains('pent house')
train['Esta_panoramica']=train.titulo.str.contains('panorámica|panoramica')
train['Esta_mirador']=train.titulo.str.contains('mirador')
train['Esta_privado']=train.titulo.str.contains('privado|privada')
train['Esta_vigilancia']=train.titulo.str.contains('vigilancia')
train['Esta_fraccionamiento']=train.titulo.str.contains('fraccionamiento')
train['Esta_refugio']=train.titulo.str.contains('refugio')
train['Esta_bancario']=train.titulo.str.contains('bancario')
train['Esta_habitarse']=train.titulo.str.contains('habitarse')

In [7]:
train[['Esta_condominio']]=train[['Esta_condominio']].fillna(value=0)
train[['Esta_loma']]=train[['Esta_loma']].fillna(value=0)
train[['Esta_bosque']]=train[['Esta_bosque']].fillna(value=0)
train[['Esta_country']]=train[['Esta_country']].fillna(value=0)
train[['Esta_lago']]=train[['Esta_lago']].fillna(value=0)
train[['Esta_residencial']]=train[['Esta_residencial']].fillna(value=0)
train[['Esta_penthouse']]=train[['Esta_penthouse']].fillna(value=0)
train[['Esta_panoramica']]=train[['Esta_panoramica']].fillna(value=0)
train[['Esta_mirador']]=train[['Esta_mirador']].fillna(value=0)
train[['Esta_privado']]=train[['Esta_privado']].fillna(value=0)
train[['Esta_vigilancia']]=train[['Esta_vigilancia']].fillna(value=0)
train[['Esta_fraccionamiento']]=train[['Esta_fraccionamiento']].fillna(value=0)
train[['Esta_refugio']]=train[['Esta_refugio']].fillna(value=0)
train[['Esta_bancario']]=train[['Esta_bancario']].fillna(value=0)
train[['Esta_habitarse']]=train[['Esta_habitarse']].fillna(value=0)

In [8]:
train['Esta_condominio'] = train['Esta_condominio'].astype(int)
train['Esta_loma'] = train['Esta_loma'].astype(int)
train['Esta_bosque'] = train['Esta_bosque'].astype(int)
train['Esta_country'] = train['Esta_country'].astype(int)
train['Esta_lago'] = train['Esta_lago'].astype(int)
train['Esta_residencial'] = train['Esta_residencial'].astype(int)
train['Esta_penthouse'] = train['Esta_penthouse'].astype(int)
train['Esta_panoramica'] = train['Esta_panoramica'].astype(int)
train['Esta_mirador'] = train['Esta_mirador'].astype(int)
train['Esta_privado'] = train['Esta_privado'].astype(int)
train['Esta_vigilancia'] = train['Esta_vigilancia'].astype(int)
train['Esta_fraccionamiento'] = train['Esta_fraccionamiento'].astype(int)
train['Esta_refugio'] = train['Esta_refugio'].astype(int)
train['Esta_bancario'] = train['Esta_bancario'].astype(int)
train['Esta_habitarse'] = train['Esta_habitarse'].astype(int)

In [9]:
train = train.drop('id', axis=1)
train = train.drop('titulo', axis=1)
train = train.drop('descripcion', axis=1)
train = train.drop('direccion', axis=1)
train = train.drop('lat', axis=1)
train = train.drop('lng', axis=1)

In [10]:
train[['antiguedad']] = train[['antiguedad']].fillna(value=8.116114)
train[['habitaciones']] = train[['habitaciones']].fillna(value=2.902326)
train[['garages']] = train[['garages']].fillna(value=1.546874)
train[['banos']] = train[['banos']].fillna(value=2.132417)
train[['metroscubiertos']] = train[['metroscubiertos']].fillna(value=174.016774)
train[['metrostotales']] = train[['metrostotales']].fillna(value=176.765145)
train[['idzona']] = train[['idzona']].fillna(value=2141183.096329)

In [11]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [12]:
train = oneHotEncoding(train,'provincia')
train = oneHotEncoding(train,'tipodepropiedad')

In [13]:
train = train.drop('Garage', axis=1)
train = train.drop('Hospedaje', axis=1)

In [14]:
#Hacemos la conversión de fechas
train['Fecha']=pd.to_datetime(train['fecha'])
train.drop(columns={'fecha'},inplace=True)

In [15]:
train['Anio']=train['Fecha'].dt.year
train = train.drop('Fecha', axis=1)

In [16]:
target_enc = ce.TargetEncoder(train['ciudad'])

# Fit the encoder using the categorical features and target
target_enc.fit(train['ciudad'], train['precio'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train['ciudad']).add_suffix('_target'))

train = train.drop('ciudad', axis=1)

In [19]:
train_label = []
train_data = []

train_label = train.iloc[:, 12]
train_data = train.iloc[:, train.columns != 'precio']

In [20]:
test = pd.read_csv('test.csv')

In [21]:
test['Esta_Capital']=test['ciudad'].apply(EstaEnCapital)

In [22]:
test['Esta_condominio']=test.titulo.str.contains('condominio')
test['Esta_loma']=test.titulo.str.contains('loma|lomas')
test['Esta_bosque']=test.titulo.str.contains('bosque')
test['Esta_country']=test.titulo.str.contains('country')
test['Esta_lago']=test.titulo.str.contains('lago')
test['Esta_residencial']=test.titulo.str.contains('residencial')
test['Esta_penthouse']=test.titulo.str.contains('pent house')
test['Esta_panoramica']=test.titulo.str.contains('panorámica|panoramica')
test['Esta_mirador']=test.titulo.str.contains('mirador')
test['Esta_privado']=test.titulo.str.contains('privado|privada')
test['Esta_vigilancia']=test.titulo.str.contains('vigilancia')
test['Esta_fraccionamiento']=test.titulo.str.contains('fraccionamiento')
test['Esta_refugio']=test.titulo.str.contains('refugio')
test['Esta_bancario']=test.titulo.str.contains('bancario')
test['Esta_habitarse']=test.titulo.str.contains('habitarse')

In [23]:
test[['Esta_condominio']]=test[['Esta_condominio']].fillna(value=0)
test[['Esta_loma']]=test[['Esta_loma']].fillna(value=0)
test[['Esta_bosque']]=test[['Esta_bosque']].fillna(value=0)
test[['Esta_country']]=test[['Esta_country']].fillna(value=0)
test[['Esta_lago']]=test[['Esta_lago']].fillna(value=0)
test[['Esta_residencial']]=test[['Esta_residencial']].fillna(value=0)
test[['Esta_penthouse']]=test[['Esta_penthouse']].fillna(value=0)
test[['Esta_panoramica']]=test[['Esta_panoramica']].fillna(value=0)
test[['Esta_mirador']]=test[['Esta_mirador']].fillna(value=0)
test[['Esta_privado']]=test[['Esta_privado']].fillna(value=0)
test[['Esta_vigilancia']]=test[['Esta_vigilancia']].fillna(value=0)
test[['Esta_fraccionamiento']]=test[['Esta_fraccionamiento']].fillna(value=0)
test[['Esta_refugio']]=test[['Esta_refugio']].fillna(value=0)
test[['Esta_bancario']]=test[['Esta_bancario']].fillna(value=0)
test[['Esta_habitarse']]=test[['Esta_habitarse']].fillna(value=0)

In [24]:
test['Esta_condominio'] = test['Esta_condominio'].astype(int)
test['Esta_loma'] = test['Esta_loma'].astype(int)
test['Esta_bosque'] = test['Esta_bosque'].astype(int)
test['Esta_country'] = test['Esta_country'].astype(int)
test['Esta_lago'] = test['Esta_lago'].astype(int)
test['Esta_residencial'] = test['Esta_residencial'].astype(int)
test['Esta_penthouse'] = test['Esta_penthouse'].astype(int)
test['Esta_panoramica'] = test['Esta_panoramica'].astype(int)
test['Esta_mirador'] = test['Esta_mirador'].astype(int)
test['Esta_privado'] = test['Esta_privado'].astype(int)
test['Esta_vigilancia'] = test['Esta_vigilancia'].astype(int)
test['Esta_fraccionamiento'] = test['Esta_fraccionamiento'].astype(int)
test['Esta_refugio'] = test['Esta_refugio'].astype(int)
test['Esta_bancario'] = test['Esta_bancario'].astype(int)
test['Esta_habitarse'] = test['Esta_habitarse'].astype(int)

In [25]:
test = test.drop('id', axis=1)
test = test.drop('titulo', axis=1)
test = test.drop('descripcion', axis=1)
test = test.drop('direccion', axis=1)
test = test.drop('lat', axis=1)
test = test.drop('lng', axis=1)

In [26]:
test[['antiguedad']] = test[['antiguedad']].fillna(value=8.116114)
test[['habitaciones']] = test[['habitaciones']].fillna(value=2.902326)
test[['garages']] = test[['garages']].fillna(value=1.546874)
test[['banos']] = test[['banos']].fillna(value=2.132417)
test[['metroscubiertos']] = test[['metroscubiertos']].fillna(value=174.016774)
test[['metrostotales']] = test[['metrostotales']].fillna(value=176.765145)
test[['idzona']] = test[['idzona']].fillna(value=2141183.096329)

In [27]:
test = oneHotEncoding(test,'provincia')
test = oneHotEncoding(test,'tipodepropiedad')

In [28]:
#Hacemos la conversión de fechas
test['Fecha']=pd.to_datetime(test['fecha'])
test.drop(columns={'fecha'},inplace=True)

In [29]:
test['Anio'] = test['Fecha'].dt.year
test = test.drop('Fecha', axis=1)

In [30]:
test = test.join(target_enc.transform(test['ciudad']).add_suffix('_target'))
test = test.drop('ciudad', axis=1)

In [33]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'target'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [34]:
#-----------------------------------------------------------------
# Modelo Random Forest Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor

In [42]:
params = [ {'n_estimators': [10, 50, 100]},]

modeloRFR = RandomForestRegressor(random_state=0, n_jobs=-1)
grid = GridSearchCV(modeloRFR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] grid search accuracy: 96.91%
[INFO] grid search best parameters: {'n_estimators': 100}


In [35]:
modeloRFR=RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=100)

In [36]:
modeloRFR.fit(train_data, train["precio"])

result = modeloRFR.predict(test)

In [37]:
write_submission("test.csv", result, "RFR_ciudadTargetE.csv")

In [84]:
#-----------------------------------------------------------------
# Modelo GradientBoostingRegressor
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor

In [15]:
params = [ {'n_estimators': [50, 100, 150]},]

modeloGBR = GradientBoostingRegressor(random_state=0)
grid = GridSearchCV(modeloGBR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] grid search accuracy: 66.77%
[INFO] grid search best parameters: {'n_estimators': 150}


In [16]:
modeloGBR=GradientBoostingRegressor(random_state=0, n_estimators=150)

In [17]:
modeloGBR.fit(train_data, train["precio"])

result = modeloGBR.predict(test)

In [18]:
write_submission("test.csv", result, "GBR.csv")

In [19]:
del modeloGBR

In [20]:
#-----------------------------------------------------------------
# Modelo Bagging Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import BaggingRegressor

In [ ]:
params = [ {'n_estimators': [10, 50, 100]},]

modeloBG = BaggingRegressor(random_state=0)
grid = GridSearchCV(modeloBG, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [ ]:
modeloBG=BaggingRegressor(random_state=0, n_jobs=-1, n_estimators=150)

In [ ]:
modeloBG.fit(train_data, train["precio"])

result = modeloBG.predict(test)

In [ ]:
write_submission("test.csv", result, "BG.csv")

In [ ]:
del modeloBG

In [11]:
#-----------------------------------------------------------------
# Modelo Multi-layer Perceptron regressor
#-----------------------------------------------------------------
from sklearn.neural_network import MLPRegressor

In [ ]:
params = [
    {
        'activation' : ('logistic', 'tanh', 'relu',),
        'solver' : ('lbfgs','adam','sgd',),
        'learning_rate' : ('constant', 'invscaling', 'adaptive',),
    },
]

modeloMLPR = MLPRegressor(random_state=0)
grid = GridSearchCV(modeloMLPR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [ ]:
modeloMLPR=MLPRegressor(random_state=0, n_estimators=150)

In [ ]:
modeloMLPR.fit(train_data, train["precio"])

result = modeloMLPR.predict(test)

In [ ]:
write_submission("test.csv", result, "MLPR.csv")